# Régression logistique sur "offers_with_10k_each_contracts"

Sans exploiter la description ni le titre des offres pour l'instant.

**PREPARATION DES DONNES**

In [180]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

In [181]:
#Import du fichier
AA = pd.read_csv("./../data/offers_with_10k_each_contracts.csv", encoding='utf-8')

In [182]:
#Répartition des contrats
AA.contract_id.value_counts()

3    10000
2    10000
1    10000
4     4800
5     3694
Name: contract_id, dtype: int64

In [183]:
#Nombre de valeurs "NaN" dans industry_id
AA.industry_id.isnull().value_counts()

False    38471
True        23
Name: industry_id, dtype: int64

In [184]:
#Nombre de valeurs "Nan" dans department_id
AA.department_id.isnull().value_counts()

False    31270
True      7224
Name: department_id, dtype: int64

In [185]:
#Nombre de valeurs "Nan" dans recruiter_id
AA.recruiter_id.isnull().value_counts()

True     28336
False    10158
Name: recruiter_id, dtype: int64

**PROBLEME :**

1/4 des valeurs pour department_id sont des valeurs "NaN". Il faut trouver une solution afin de ne pas perdre d'informations et de données. Cela semble moins dérangeant pour recruiter_id. A priori on ne prédit pas le contrat en fonction de l'entreprise qui recrute (quoi que ?). 

Une solution serait de remplacer les valeurs manquantes dans chaque colonne par les valeurs les plus fréquentes. Cela n'empêche pas que pour department_id, 1/4 des valeurs restent "faussées".

Donc : Est-ce vraiment pertinent ? Maybe, maybe not.

In [186]:
from sklearn.impute import SimpleImputer

In [187]:
AA = AA.sample(frac=1).reset_index(drop=True)
AA = AA.head(6000)                    #Selection des 500 premières lignes mélangées au préalable sinon trop long à exécuter

#Ne fonctionne pas
#imp = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
#imp.fit_transform(AA)

AA['target'] = AA['contract_id']     #Création d'une nouvelle colonne target qui nous sera utile par la suite

In [188]:
AA

,offer_id,title,description,contract_id,recruiter_id,department_id,industry_id,location,latitude,longitude,created_at,target
0,316757,Concepteur / Développeur SOA webMethods H/F,MISSION \nVous êtes concepteur/réalisateur/con...,1,NaN,8.0,17.0,"LILLE, France",50.629250,3.057256,2015-03-25 17:27:52.307554,1
1,363425,Chargé d'affaires H/F,Manpower Conseil Recrutement recherche pour so...,1,NaN,1.0,20.0,"Montrond les bains, France",45.639301,4.238760,2015-05-21 09:31:41.573613,1
2,6573,H/F Responsable des Opérations,"Entreprise :\n\nPour un de nos clients, un act...",5,NaN,109.0,15.0,"Paris, France",-8.811036,115.103803,2014-07-08 08:45:01.132966,5
3,7902,CHEF DE PRODUIT MARKETING H/F,Description de l'offre : \nDemos recherche pou...,4,25.0,111.0,24.0,"Île-de-France, France",-8.811036,115.103803,2014-07-08 08:50:41.772186,4
4,340141,"Chargé(e) d'étude ""promotion des produits sous...",Entreprise :\n\nCREPAL (comité régional de pro...,2,NaN,104.0,19.0,"Limousin, France",45.774109,1.714179,2015-04-15 20:24:03.292594,2
5,374599,H/F Ingénieur infrastructure,Entreprise :\n\n\nSynopsys Emulation and Verif...,1,NaN,7.0,15.0,"Wissous, France",48.743500,2.317900,2015-05-21 19:13:17.625357,1
6,681522,Manager de rayons PGC (H/F),"Résumé\nManager de rayons PGC H/F, 75014 Paris...",1,23069.0,101.0,10.0,"Paris, France",48.832770,2.324870,2018-07-10 20:01:23.366717,1
7,386730,TECHNICIEN SYSTEMES RESEAUX TELEPHONIE H/F,Entreprise :\n\nDirection : Systèmes d'inform...,2,NaN,8.0,22.0,"ALIXAN, France",44.974980,5.026462,2015-05-22 05:35:17.009325,2
8,362258,Chargé d'affaires commercial tuyauterie/fluide...,"JSB & Brothers, cabinet de conseil en ressourc...",1,NaN,1.0,15.0,"Proche Poissy, France",46.637280,2.338262,2015-05-21 08:31:17.881269,1
9,7165,CHARGE D AFFAIRES EN ELECTRICITE DEBUTANT H/F,Entreprise :\n\nVous recherchez une entreprise...,4,NaN,110.0,22.0,"Ajaccio, France",-8.811036,115.103803,2014-07-08 08:46:23.530669,4


In [189]:
#Remplacement des missing values par la valeur la plus fréquente pour chaque colonne
AA = AA.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [190]:
AA

,offer_id,title,description,contract_id,recruiter_id,department_id,industry_id,location,latitude,longitude,created_at,target
0,316757,Concepteur / Développeur SOA webMethods H/F,MISSION \nVous êtes concepteur/réalisateur/con...,1,23069.0,8.0,17.0,"LILLE, France",50.629250,3.057256,2015-03-25 17:27:52.307554,1
1,363425,Chargé d'affaires H/F,Manpower Conseil Recrutement recherche pour so...,1,23069.0,1.0,20.0,"Montrond les bains, France",45.639301,4.238760,2015-05-21 09:31:41.573613,1
2,6573,H/F Responsable des Opérations,"Entreprise :\n\nPour un de nos clients, un act...",5,23069.0,109.0,15.0,"Paris, France",-8.811036,115.103803,2014-07-08 08:45:01.132966,5
3,7902,CHEF DE PRODUIT MARKETING H/F,Description de l'offre : \nDemos recherche pou...,4,25.0,111.0,24.0,"Île-de-France, France",-8.811036,115.103803,2014-07-08 08:50:41.772186,4
4,340141,"Chargé(e) d'étude ""promotion des produits sous...",Entreprise :\n\nCREPAL (comité régional de pro...,2,23069.0,104.0,19.0,"Limousin, France",45.774109,1.714179,2015-04-15 20:24:03.292594,2
5,374599,H/F Ingénieur infrastructure,Entreprise :\n\n\nSynopsys Emulation and Verif...,1,23069.0,7.0,15.0,"Wissous, France",48.743500,2.317900,2015-05-21 19:13:17.625357,1
6,681522,Manager de rayons PGC (H/F),"Résumé\nManager de rayons PGC H/F, 75014 Paris...",1,23069.0,101.0,10.0,"Paris, France",48.832770,2.324870,2018-07-10 20:01:23.366717,1
7,386730,TECHNICIEN SYSTEMES RESEAUX TELEPHONIE H/F,Entreprise :\n\nDirection : Systèmes d'inform...,2,23069.0,8.0,22.0,"ALIXAN, France",44.974980,5.026462,2015-05-22 05:35:17.009325,2
8,362258,Chargé d'affaires commercial tuyauterie/fluide...,"JSB & Brothers, cabinet de conseil en ressourc...",1,23069.0,1.0,15.0,"Proche Poissy, France",46.637280,2.338262,2015-05-21 08:31:17.881269,1
9,7165,CHARGE D AFFAIRES EN ELECTRICITE DEBUTANT H/F,Entreprise :\n\nVous recherchez une entreprise...,4,23069.0,110.0,22.0,"Ajaccio, France",-8.811036,115.103803,2014-07-08 08:46:23.530669,4


**REGRESSION LOGISTIQUE**

Testons la regression logistique sur nos données avec pour features department_id, industry_id. 

In [191]:
from sklearn.model_selection import train_test_split

In [192]:
feature = ["department_id" , "industry_id"]

features = AA[feature] 
target = AA['target']

#AA = AA.drop(['offer_id', 'title', 'description','location', 'latitude', 'longitude', 'created_at'], axis=1)

In [193]:
AA.head()

,offer_id,title,description,contract_id,recruiter_id,department_id,industry_id,location,latitude,longitude,created_at,target
0,316757,Concepteur / Développeur SOA webMethods H/F,MISSION \nVous êtes concepteur/réalisateur/con...,1,23069.0,8.0,17.0,"LILLE, France",50.629250,3.057256,2015-03-25 17:27:52.307554,1
1,363425,Chargé d'affaires H/F,Manpower Conseil Recrutement recherche pour so...,1,23069.0,1.0,20.0,"Montrond les bains, France",45.639301,4.238760,2015-05-21 09:31:41.573613,1
2,6573,H/F Responsable des Opérations,"Entreprise :\n\nPour un de nos clients, un act...",5,23069.0,109.0,15.0,"Paris, France",-8.811036,115.103803,2014-07-08 08:45:01.132966,5
3,7902,CHEF DE PRODUIT MARKETING H/F,Description de l'offre : \nDemos recherche pou...,4,25.0,111.0,24.0,"Île-de-France, France",-8.811036,115.103803,2014-07-08 08:50:41.772186,4
4,340141,"Chargé(e) d'étude ""promotion des produits sous...",Entreprise :\n\nCREPAL (comité régional de pro...,2,23069.0,104.0,19.0,"Limousin, France",45.774109,1.714179,2015-04-15 20:24:03.292594,2


In [194]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.7)

In [195]:
from collections import Counter
Counter(target_test)

Counter({1: 1122, 2: 1095, 3: 1055, 4: 545, 5: 383})

In [196]:
#Régression logistique

cls = LogisticRegression() 
cls.fit(features_train, target_train)
predictions = cls.predict(features_test)
#cls.intercept_
#cls.coef_
print('Matrice de confusion :\n', confusion_matrix(target_test, predictions))

Matrice de confusion :
 [[380 385 330  27   0]
 [226 663 156  50   0]
 [ 79 183 786   7   0]
 [  1 454  45  45   0]
 [  0 326  29  28   0]]


C:\Users\Marine\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Marine\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [197]:
print(classification_report(cls.predict(features_test), target_test))

              precision    recall  f1-score   support

           1       0.34      0.55      0.42       686
           2       0.61      0.33      0.43      2011
           3       0.75      0.58      0.65      1346
           4       0.08      0.29      0.13       157
           5       0.00      0.00      0.00         0

   micro avg       0.45      0.45      0.45      4200
   macro avg       0.35      0.35      0.33      4200
weighted avg       0.59      0.45      0.49      4200



C:\Users\Marine\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


**CONCLUSION**

Comme on pouvait s'y attendre, les performances sont assez médiocres, contrairement à ce que l'on peut obtenir avec comme variable explicative uniquement les descriptions de poste (ce qui a été fait parallèlement dans un notebook). 

**Piste d'amélioration**
Une idée pour améliorer notre classification serait d'ajouter d'une part les descriptions de postes à nos 2 variables explicatives (que sont industry_id et department_id), et d'autre part les titres d'offre. 